# import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Packages

import pandas as pd
import numpy as np
import scipy.sparse as sps
import time

%cd /content/drive/My Drive/GitHub/minor-goodbooks-recommender

/content/drive/My Drive/Packages
/content/drive/My Drive/GitHub/minor-goodbooks-recommender


In [3]:
ratings_random = pd.read_csv('data/ratings_random.csv')
ratings_random

,user_id,book_id,rating
0,58,26,1
1,58,58,4
2,58,18,5
3,58,14,3
4,58,2,3
...,...,...,...
298171,17635,1922,5
298172,10622,6810,4
298173,10622,610,5
298174,53358,253,5


# simplest similatrity

In [ ]:
ratings_s = sps.csr_matrix(ratings_random.pivot('user_id', 'book_id', 'rating').fillna(0).astype(int))

In [ ]:
from sklearn.neighbors import NearestNeighbors

simple_knn = NearestNeighbors(metric='cosine', n_neighbors=10)
simple_knn.fit(ratings_s.transpose())

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [ ]:
sim = simple_knn.kneighbors(ratings_s.transpose())

In [ ]:
indices = sim[1].flatten()
data = sim[0].flatten()
indptr = np.array(list(range(0, 99901, 10)))

In [ ]:
simmatrix = sps.csr_matrix((data, indices, indptr))
simmatrix.view()

In [ ]:
import pickle

In [ ]:
simmatrix

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9950,9951,9952,9953,9954,9955,9956,9957,9958,9959,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989
0,2.087219e-14,3.934602e-01,0.446234,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.455517,0.000000,0.000000,0.0,0.542293,0.275365,0.000000,0.0,0.332395,0.541218,0.000000,0.000000,0.000000,0.529628,0.0,0.000000,0.000000,0.000000,0.0,0.530372,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3.934602e-01,1.865175e-14,0.458794,0.000000,0.000000e+00,0.0,0.493093,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.322214,0.0,0.000000,0.354208,0.000000,0.328678,0.335112,0.358426,0.0,0.356575,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.462337e-01,4.587942e-01,0.000000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.573228,0.000000,0.0,0.000000,0.000000,0.584646,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5.491609e-01,4.943351e-01,0.000000,0.000000,4.181255e-01,0.0,0.000000,0.453589,0.0,0.517693,0.0,0.000000,0.535763,0.510562,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.503888,0.000000,0.0,0.000000,0.506956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000e+00,5.366495e-01,0.000000,0.418126,6.661338e-16,0.0,0.000000,0.456355,0.0,0.537101,0.0,0.000000,0.513781,0.532265,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.523602,0.562083,0.0,0.000000,0.518455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def view_csr(self):
  return pd.DataFrame.sparse.from_spmatrix(self).head()

sps.csr_matrix.view = view_csr

In [ ]:
rating_vector = ratings_s[0, :]
rating_vector.view()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,9950,9951,9952,9953,9954,9955,9956,9957,9958,9959,9960,9961,9962,9963,9964,9965,9966,9967,9968,9969,9970,9971,9972,9973,9974,9975,9976,9977,9978,9979,9980,9981,9982,9983,9984,9985,9986,9987,9988,9989
0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0


In [ ]:
rated = rating_vector.view().transpose()
rated = rated.loc[rated[0] != 0].index
rated

Int64Index([  10,   15,   29,   30,   32,   46,   57,   59,   60,   63,   66,
              94,   99,  114,  126,  139,  142,  143,  145,  194,  210,  211,
             255,  257,  266,  290,  312,  319,  358,  395,  406,  458,  477,
             519,  532,  550,  619,  672,  688,  696,  701,  821,  824,  835,
             849,  868,  905,  915,  929,  968,  982, 1010, 1011, 1075, 1172,
            1186, 1211, 1251, 1407, 1521, 1687, 1699, 1711, 1757, 1881, 1891,
            2036, 2106, 2136, 2255, 2283, 2554, 2834, 2836, 3009, 3047, 3426,
            3586, 3646, 4211, 4723, 5042, 5089, 5190, 5239, 5288, 6444, 6721,
            6835, 6988, 8048, 8063, 8111, 9739, 9973, 9979],
           dtype='int64')

In [ ]:
pred = (rating_vector * simmatrix)
pred_list = pred.wiev().transpose().sort_values(by=0, ascending = False).reset_index()
pred_list = pred_list.loc[~pred_list['index'].isin(rated), :].head(10)
pred_list.columns = ['book_id', 'predicted_rating']
pred_list

,book_id,predicted_rating
1,45,38.856039
9,667,28.748925
10,21,27.751885
14,37,22.347972
15,56,22.291289
17,234,20.981811
21,4246,19.129415
27,317,16.412955
28,80,16.153339
29,261,16.028705


In [ ]:
def recommend(rating_vector, simmatrix):
  rated = rating_vector.view().transpose()
  rated = rated.loc[rated[0] != 0].index

  pred = (rating_vector * simmatrix)
  pred_list = pred.wiev().transpose().sort_values(by=0, ascending = False).reset_index()
  pred_list = pred_list.loc[~pred_list['index'].isin(rated), :].head(10)
  pred_list.columns = ['book_id', 'predicted_rating']

  return pred_list

In [ ]:
recommend(ratings_s[11, :], simmatrix)

,book_id,predicted_rating
0,3,48.934689
1,4,42.566867
12,114,23.900268
14,12,20.135447
20,13,18.078588
22,1158,17.686774
29,64,14.892952
30,31,14.789587
32,170,14.174234
34,28,13.370019


# need to retrain anyway!

In [5]:
!pip install surprise

     |████████████████████████████████| 11.8MB 8.3MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1617584 sha256=ff64c29ea8c6d5869af7935a11ad509a9dd27d3e24df66b804bda5114b5036ad
  Stored in directory: /root/.cache/pip/wheels/78/9c/3d/41b419c9d2aff5b6e2b4c0fc8d25c538202834058f9ed110d0
Successfully built scikit-surprise


In [13]:
%cd /content/drive/My Drive/Packages

from surprise import Reader, Dataset
from surprise import KNNWithMeans, SVD, SVDpp

%cd /content/drive/My Drive/GitHub/minor-goodbooks-recommender

/content/drive/My Drive/Packages
/content/drive/My Drive/GitHub/minor-goodbooks-recommender


In [8]:
ratings = pd.read_csv('data/ratings.csv')
user_ratings = ratings.loc[ratings['user_id'] == 1]

In [9]:
books = pd.read_csv('data/books.csv')
book_map = books[['id', 'title']]

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_random, reader)
data_full = Dataset.load_from_df(ratings_random.append(user_ratings), reader).build_full_trainset()

In [10]:
def recommend_list(user_ratings, ratings_data, algorithm, verbose = False):
  start = time.time()

  reader = Reader(rating_scale=(1, 5))
  data = Dataset.load_from_df(ratings_data, reader)
  data_full = Dataset.load_from_df(ratings_data.append(user_ratings), reader).build_full_trainset()

  algorithm.fit(data_full)

  preds = []
  for i in range(1, 9901):
    preds.append(algorithm.predict(user_ratings.user_id.unique()[0], i).est)
  
  recs = pd.DataFrame({'book_id' : range(1,9901), 'estimated_rating' : preds})
  recs = recs.loc[~recs['book_id'].isin(user_ratings['book_id'])].sort_values('estimated_rating', ascending = False).head(10)
  if verbose:
    print('Seconds: ', time.time()-start)

  return recs

In [11]:
recommend_list(ratings.loc[ratings['user_id'] == 1], ratings_random, knn, verbose=True)

NameError: ignored

In [15]:
svd = SVD(n_factors=20, verbose=False)
r = recommend_list(ratings.loc[ratings['user_id'] == 1], ratings_random, svd, verbose=True)

Seconds:  9.482113122940063


In [21]:
book_map

,id,title
0,1,"The Hunger Games (The Hunger Games, #1)"
1,2,Harry Potter and the Sorcerer's Stone (Harry P...
2,3,"Twilight (Twilight, #1)"
3,4,To Kill a Mockingbird
4,5,The Great Gatsby
...,...,...
9995,9996,"Bayou Moon (The Edge, #2)"
9996,9997,"Means of Ascent (The Years of Lyndon Johnson, #2)"
9997,9998,The Mauritius Command
9998,9999,Cinderella Ate My Daughter: Dispatches from th...


In [22]:
rec_table = pd.merge(r, book_map.rename({'id' : 'book_id'}, axis=1), on='book_id')

In [23]:
rec_table

,book_id,estimated_rating,title
0,3628,4.615239,The Complete Calvin and Hobbes
1,1808,4.525431,"Morning Star (Red Rising, #3)"
2,780,4.410230,Calvin and Hobbes
3,862,4.387412,"Words of Radiance (The Stormlight Archive, #2)"
4,1308,4.386483,A Court of Mist and Fury (A Court of Thorns an...
5,2840,4.381215,Homegoing
6,7593,4.376251,"This Is Water: Some Thoughts, Delivered on a S..."
7,9141,4.368599,"The Way of Kings, Part 1 (The Stormlight Archi..."
8,1788,4.360621,The Calvin and Hobbes Tenth Anniversary Book
9,3241,4.356755,"Crooked Kingdom (Six of Crows, #2)"
